#Загрузка дата сета и выгрузка данных из zip в подходящие директории

In [1]:
import urllib.request
import zipfile

In [7]:
url = "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip"

In [8]:
file_name ='horse-or-human.zip'
training_dir = 'horse-or-human/training/'
urllib.request.urlretrieve(url, file_name)

('horse-or-human.zip', <http.client.HTTPMessage at 0x7f726cc91b90>)

In [9]:
zip_ref = zipfile.ZipFile(file_name, 'r')
zip_ref.extractall(training_dir)
zip_ref.close()

#Теперь запихнем все в TensorFlow via ImageDataGenerator, чтобы автоматически создать labels

In [11]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [12]:
train_datagen = ImageDataGenerator(rescale=1/255) #создаем генератор

In [13]:
train_generator = train_datagen.flow_from_directory(training_dir, target_size=(300, 300), class_mode='binary')

Found 1027 images belonging to 2 classes.


#Создаем модель

**Сейчас стоит задача бинарной классификации, потому на выходе достаточно одного нейрона, который дает 0 или 1 в зависимости от принадлежности к тому или иному классу**

In [14]:
import tensorflow as tf

In [19]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(300, 300, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_15 (Conv2D)          (None, 298, 298, 16)      448       
                                                                 
 max_pooling2d_15 (MaxPoolin  (None, 149, 149, 16)     0         
 g2D)                                                            
                                                                 
 conv2d_16 (Conv2D)          (None, 147, 147, 32)      4640      
                                                                 
 max_pooling2d_16 (MaxPoolin  (None, 73, 73, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_17 (Conv2D)          (None, 71, 71, 64)        18496     
                                                                 
 max_pooling2d_17 (MaxPoolin  (None, 35, 35, 64)       0

In [22]:
from tensorflow.keras.optimizers import RMSprop

In [24]:
model.compile(loss='binary_crossentropy', optimizer=RMSprop(learning_rate=0.001), metrics=['accuracy'])

In [25]:
history = model.fit_generator(train_generator, epochs=10)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  """Entry point for launching an IPython kernel.


Epoch 1/10
33/33 [==============================] - 88s 3s/step - loss: 0.7353 - accuracy: 0.6456
Epoch 2/10
33/33 [==============================] - 86s 3s/step - loss: 0.3048 - accuracy: 0.9182
Epoch 3/10
33/33 [==============================] - 92s 3s/step - loss: 0.1588 - accuracy: 0.9416
Epoch 4/10
33/33 [==============================] - 93s 3s/step - loss: 0.2126 - accuracy: 0.9503
Epoch 5/10
33/33 [==============================] - 94s 3s/step - loss: 0.0274 - accuracy: 0.9873
Epoch 6/10
33/33 [==============================] - 94s 3s/step - loss: 0.2153 - accuracy: 0.9601
Epoch 7/10
33/33 [==============================] - 90s 3s/step - loss: 0.0766 - accuracy: 0.9708
Epoch 8/10
33/33 [==============================] - 88s 3s/step - loss: 0.0033 - accuracy: 1.0000
Epoch 9/10
33/33 [==============================] - 86s 3s/step - loss: 1.4853e-04 - accuracy: 1.0000
Epoch 10/10
33/33 [==============================] - 87s 3s/step - loss: 1.9330e-05 - accuracy: 1.0000


#Добавим валидацию

In [27]:
validation_url = "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip"

In [28]:
validation_file_name ="valudation-horse-or-human.zip"
validation_dir = "horse-or-human/validation/"
urllib.request.urlretrieve(validation_url, validation_file_name)

('valudation-horse-or-human.zip', <http.client.HTTPMessage at 0x7f71eeea3950>)

In [29]:
zip_ref = zipfile.ZipFile(validation_file_name, 'r')
zip_ref.extractall(validation_dir)
zip_ref.close()

**Создаем генератор для валидации**

In [44]:
validation_datagen = ImageDataGenerator(rescale=1/255)

validation_generator = train_datagen.flow_from_directory(
    validation_dir,
    target_size=(300, 300),
    class_mode='binary'
)

Found 256 images belonging to 2 classes.


сделаем новую модель но с вализацией

In [ ]:
#копируем модель и запускаем обучение
#нужно это реализовать

In [48]:
history = model.fit_generator(train_generator, epochs=3 , validation_data = validation_generator)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  """Entry point for launching an IPython kernel.


Epoch 1/3
33/33 [==============================] - 100s 3s/step - loss: 4.3818e-04 - accuracy: 1.0000 - val_loss: 1.7850 - val_accuracy: 0.8789
Epoch 2/3
33/33 [==============================] - 94s 3s/step - loss: 0.1080 - accuracy: 0.9844 - val_loss: 1.9582 - val_accuracy: 0.7891
Epoch 3/3
33/33 [==============================] - 97s 3s/step - loss: 0.0016 - accuracy: 1.0000 - val_loss: 3.1722 - val_accuracy: 0.7930


In [49]:
classes = model.predict(validation_generator)

In [50]:
for item in classes:
  if item > 0.5:
    print('Human')
  else:
    print('Horse')

Horse
Human
Human
Horse
Human
Human
Human
Horse
Human
Human
Human
Human
Human
Horse
Human
Human
Horse
Horse
Human
Horse
Human
Human
Human
Human
Human
Horse
Human
Horse
Horse
Horse
Horse
Human
Human
Human
Horse
Horse
Human
Human
Human
Horse
Human
Horse
Human
Human
Human
Horse
Human
Human
Human
Human
Horse
Human
Horse
Human
Horse
Human
Human
Horse
Human
Human
Human
Horse
Horse
Horse
Human
Horse
Horse
Horse
Horse
Human
Horse
Horse
Human
Human
Human
Human
Human
Human
Horse
Human
Human
Human
Human
Human
Human
Human
Horse
Horse
Human
Horse
Human
Human
Horse
Horse
Horse
Human
Human
Human
Horse
Human
Horse
Horse
Human
Human
Human
Horse
Human
Horse
Human
Human
Horse
Human
Human
Human
Horse
Human
Human
Horse
Human
Horse
Human
Human
Horse
Horse
Human
Human
Horse
Human
Horse
Horse
Human
Human
Human
Human
Human
Horse
Human
Human
Human
Human
Human
Horse
Human
Human
Human
Horse
Human
Horse
Human
Horse
Human
Human
Human
Horse
Human
Human
Human
Horse
Human
Human
Human
Human
Horse
Horse
Human
Human
Huma